In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2

# 데이터 불러오기
my_images = []
labels = [0]*5 + [1]*10  # 타인=0, 내얼굴=1

for i in range(15):
    file = f"./my_images/img{i + 1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    my_images.append(image)

X = np.array(my_images, dtype='float32') / 255.0
y = np.array(labels)

# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
datagen.fit(X_train)

# ✅ MobileNetV2 전이 학습 모델 사용
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96,96,3))
base_model.trainable = False  # 특징 추출부 고정

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=2),
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=[early_stop],
    verbose=1
)

model.save("MY_FACE_MODEL.keras")

# 테스트
test_images = []
for i in range(10):
    file = f"./test_images/img{i + 1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    test_images.append(image)

X_test = np.array(test_images, dtype='float32') / 255.0
cnn_model = keras.models.load_model("MY_FACE_MODEL.keras")

predictions = cnn_model.predict(X_test)

for i, p in enumerate(predictions):
    label = "✅ 내 얼굴" if p > 0.5 else "❌ 타인"
    print(f"img{i+1:02d}.jpg → {label} ({p[0]:.3f})")

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 393ms/step - accuracy: 0.5000 - loss: 0.7147 - val_accuracy: 0.6000 - val_loss: 0.6874
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5000 - loss: 0.7852 - val_accuracy: 0.6000 - val_loss: 0.6927
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.5000 - loss: 0.7477 - val_accuracy: 0.6000 - val_loss: 0.6973
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.6000 - loss: 0.7722 - val_accuracy: 0.6000 - val_loss: 0.6983
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.6000 - loss: 0.6282 - val_accuracy: 0.6000 - val_loss: 0.6999
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.8000 - loss: 0.5610 - val_accuracy: 0.6000 - val_loss: 0.7052
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.7000 - loss: 0.5357 - val_accuracy: 0.6000 - val_loss: 0.7126
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.8000 - loss: 0.7012 - val_accuracy: 0.6000 - va